In [1]:
%pip install -r requirements.txt

  Cloning https://github.com/microsoft/qlib.git to /tmp/pip-install-cqhhg_sd/pyqlib_2262f277289b486f9e42df8d70c164cc
  Running command git clone --filter=blob:none --quiet https://github.com/microsoft/qlib.git /tmp/pip-install-cqhhg_sd/pyqlib_2262f277289b486f9e42df8d70c164cc
  Resolved https://github.com/microsoft/qlib.git to commit 98f569eed2252cc7fad0c120cad44f6181c3acf6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 10.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 8.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Suc

In [2]:
import os

if not os.path.exists("data"):
    os.mkdir("data")

In [3]:
import yahooquery as yq

ticker_symbol = "AAPL"
stock = yq.Ticker(ticker_symbol)
stock_df = stock.history(period="max", end="2023-11-20")
stock_df.to_csv("data/aapl.csv", encoding="UTF-8")

In [4]:
!python scripts/csv2qlib.py dump_all --csv_path data --qlib_dir data/qlib_data --include_fields open,high,low,close,volume,adjclose

2023-11-23 06:51:05.584 | INFO     | __main__:_get_all_date:275 - start get all date......
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 16.22it/s]
2023-11-23 06:51:05.646 | INFO     | __main__:_get_all_date:294 - end of get all date.

2023-11-23 06:51:05.646 | INFO     | __main__:_dump_calendars:297 - start dump calendars......
2023-11-23 06:51:05.675 | INFO     | __main__:_dump_calendars:300 - end of calendars dump.

2023-11-23 06:51:05.675 | INFO     | __main__:_dump_instruments:303 - start dump instruments......
2023-11-23 06:51:05.676 | INFO     | __main__:_dump_instruments:305 - end of instruments dump.

2023-11-23 06:51:05.676 | INFO     | __main__:_dump_features:308 - start dump features......
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 13.79it/s]
2023-11-23 06:51:05.748 | INFO     | __main__:_dump_features:315 - end of features dump.



In [5]:
import qlib
from qlib.constant import REG_US
from qlib.data import D

provider_uri = "data/qlib_data"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_US)

def get_aapl_dataset_df(input_dict, start_time="1999-12-01", end_time="2023-11-10", freq="day"):
    fields = ["$open", "$high", "$low", "$close", "$volume", "$adjclose"]
    df = D.features(["AAPL"], input_dict.values(), start_time=start_time, end_time=end_time, freq=freq).rename(columns={v: k for k, v in input_dict.items()})
    return df

fields_dict = {
    "Residual": "$adjclose-Ref($adjclose,1)",  # equal to "Delta": "Delta($adjclose,1)",
    "Open": "$open",
    "High": "$high",
    "Low": "$low",
    "Close": "$close",
    "Adj_Close": "$adjclose",
    "Factor": "$adjclose/$close",
    "Volume": "$volume",
}
basic = get_aapl_dataset_df(fields_dict)
basic = basic.loc["AAPL"]["2000-01-01":"2022-12-31"]
basic.to_csv("data/aapl_qlib.csv")

[42189:MainThread](2023-11-23 06:51:06,091) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[42189:MainThread](2023-11-23 06:51:06,244) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[42189:MainThread](2023-11-23 06:51:06,244) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/home/ziyang/Projects/DLF/A3/data/qlib_data')}


In [6]:
# Feature Engineering
fields_dict = {
    "Residual": "$adjclose-Ref($adjclose,1)",  # equal to "Delta": "Delta($adjclose,1)",
    "Open": "$open",
    "High": "$high",
    "Low": "$low",
    "Close": "$close",
    "Adj_Close": "$adjclose",
    "Factor": "$adjclose/$close",
    "Volume": "$volume",
    "Open_Close_Diff": "$open-$close",
    "Open_AdjClose_Diff": "$open-$adjclose",
    "High_Low_Diff": "$high-$low",
    "DIF": "(EMA($close, 12) - EMA($close, 26))/$close",
    "AdjDIF": "(EMA($adjclose, 12) - EMA($adjclose, 26))/$adjclose",
    "DEA": "EMA((EMA($close, 12) - EMA($close, 26))/$close, 9) ",
    "AdjDEA": "EMA((EMA($adjclose, 12) - EMA($adjclose, 26))/$adjclose, 9) ",
    "MACD": "(EMA($close, 12) - EMA($close, 26))/$close - EMA((EMA($close, 12) - EMA($close, 26))/$close, 9)/$close",
    "AdjMACD": "(EMA($adjclose, 12) - EMA($adjclose, 26))/$adjclose - EMA((EMA($adjclose, 12) - EMA($adjclose, 26))/$adjclose, 9)/$adjclose",
}

for ops in ["Mean", "EMA", "WMA"]:
    for f in ["$open", "$high", "$low", "$close", "$adjclose"]:
        for n in [1, 3, 5, 10, 20]:
            fields_dict[f"{f[1:].title()}_{'MA' if ops=='Mean' else ops}{n}"] = f"{ops}({f}, {n})"

for ops in ["Skew", "Kurt", "Mad", "Slope", "Rsquare", "Resi"]:
    for f in ["$open", "$high", "$low", "$close", "$adjclose"]:
        for n in [3, 5, 10, 20]:
            if ops == "Kurt" and n < 5:
                continue
            fields_dict[f"{f[1:].title()}_{ops}{n}"] = f"{ops}({f}, {n})"

label = "Residual"
label_idx = list(fields_dict.keys()).index(label)
feats_len = len(fields_dict)
data = get_aapl_dataset_df(fields_dict)
data.isna().sum().sum()

1

In [7]:
def rsi(values):
    up = values[values > 0].mean()
    down = -1 * values[values < 0].mean()
    return 100 * up / (up + down)

In [8]:
data["Close_Momentum1"] = (data["Close"] - data["Close"].shift(1)).fillna(0)
data["Close_Momentum3"] = (data["Close"] - data["Close"].shift(3)).fillna(0)
data["Close_Momentum5"] = (data["Close"] - data["Close"].shift(5)).fillna(0)
data["Close_Momentum10"] = (data["Close"] - data["Close"].shift(10)).fillna(0)
data["Close_Momentum20"] = (data["Close"] - data["Close"].shift(20)).fillna(0)
data["Close_RSI14"] = data["Close_Momentum1"].rolling(center=False, window=14).apply(rsi).fillna(0)
data["AdjClose_Momentum1"] = (data["Adj_Close"] - data["Adj_Close"].shift(1)).fillna(0)
data["AdjClose_Momentum3"] = (data["Adj_Close"] - data["Adj_Close"].shift(3)).fillna(0)
data["AdjClose_Momentum5"] = (data["Adj_Close"] - data["Adj_Close"].shift(5)).fillna(0)
data["AdjClose_Momentum10"] = (data["Adj_Close"] - data["Adj_Close"].shift(10)).fillna(0)
data["AdjClose_Momentum20"] = (data["Adj_Close"] - data["Adj_Close"].shift(20)).fillna(0)
data["AdjClose_RSI14"] = data["AdjClose_Momentum1"].rolling(center=False, window=14).apply(rsi).fillna(0)

In [9]:
import numpy as np

def bollingerbands(price, length=21, num_sd=2):
    """returns middle, upper band, and lower band"""
    mid = price.rolling(window=length, center=False).mean()
    sd = price.rolling(window=length, center=False).std()
    upperband = mid + (sd * num_sd)
    lowerband = mid - (sd * num_sd)
    return np.round(mid, 3), np.round(upperband, 3), np.round(lowerband, 3)

In [10]:
data["Close_BB_Middle_Band"], data["Close_BB_Upper_Band"], data["Close_BB_Lower_Band"] = bollingerbands(data["Close"], length=21, num_sd=2)
data["AdjClose_BB_Middle_Band"], data["AdjClose_BB_Upper_Band"], data["AdjClose_BB_Lower_Band"] = bollingerbands(data["Adj_Close"], length=21, num_sd=2)

In [11]:
# Calculation of Average Directional Movement Index (ADX)
def DMI(df, period):
    df1 = df.copy()
    df1["highlow"] = df1["High"] - df1["High"]
    df1["abshighclose"] = abs(df1["High"] - df1["Close"].shift(1))
    df1["abslowclose"] = abs(df1["Low"] - df1["Close"].shift(1))

    df1["UpMove"] = df1["High"] - df1["High"].shift(1)
    df1["DownMove"] = df1["Low"].shift(1) - df1["Low"]
    df1["Zero"] = 0

    df1["DM_plus"] = np.where((df1["UpMove"] > df1["DownMove"]) & (df1["UpMove"] > df1["Zero"]), df1["UpMove"], 0)
    df1["DM_minus"] = np.where((df1["UpMove"] < df1["DownMove"]) & (df1["DownMove"] > df1["Zero"]), df1["DownMove"], 0)
    df1["TR"] = df1[["highlow", "abshighclose", "abslowclose"]].max(axis=1)
    df["ATR"] = df1["TR"].rolling(window=14).mean()

    df["DI_plus"] = 100 * (df1["DM_plus"] / df["ATR"]).ewm(span=period, min_periods=0, adjust=True, ignore_na=False).mean()
    df["DI_minus"] = 100 * (df1["DM_minus"] / df["ATR"]).ewm(span=period, min_periods=0, adjust=True, ignore_na=False).mean()

    df["ADX"] = 100 * (abs((df["DI_plus"] - df["DI_minus"]) / (df["DI_plus"] + df["DI_minus"]))).ewm(span=period, min_periods=0, adjust=True, ignore_na=False).mean()

DMI(data, 14)

data["pct"] = data["Close"].pct_change()

In [12]:
df = data.loc["AAPL"]["2000-01-01":"2022-12-31"]
print(df.isna().sum().sum())
df = df.fillna(0)
df.to_csv("data/feat_eng_raw.csv")
df.isna().sum().sum()

1


0

In [13]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(100, 100))
sns.heatmap(data.corr(), cmap="crest")
plt.tight_layout()
plt.show()

In [14]:
# Feature Selection 1
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, r_regression, f_regression, mutual_info_regression
from sklearn.preprocessing import MinMaxScaler

train_x = df.copy(deep=True).values
train_y = df.copy(deep=True)[label].values
train_y_2 = df.copy(deep=True)["Adj_Close"].values

train_x = MinMaxScaler().fit_transform(train_x)
train_y = MinMaxScaler().fit_transform(train_y.reshape(-1, 1))
train_y_2 = MinMaxScaler().fit_transform(train_y_2.reshape(-1, 1))
#############################################################################################
rr_model = SelectKBest(r_regression)
rr_model.fit(train_x, train_y.ravel())
rr_filter = data.columns.values[np.argsort(rr_model.scores_)[:50]]

rr_model = SelectKBest(r_regression)
rr_model.fit(train_x, train_y_2.ravel())
rr_filter_2 = data.columns.values[np.argsort(rr_model.scores_)[:50]]
rr_filter = np.union1d(rr_filter, rr_filter_2)
#############################################################################################
fr_model = SelectKBest(f_regression)
fr_model.fit(train_x, train_y.ravel())
fr_filter = data.columns.values[np.argsort(fr_model.scores_)[:50]]

fr_model = SelectKBest(f_regression)
fr_model.fit(train_x, train_y_2.ravel())
fr_filter_2 = data.columns.values[np.argsort(fr_model.scores_)[:50]]
fr_filter = np.union1d(fr_filter, fr_filter_2)
#############################################################################################
mir_model = SelectKBest(mutual_info_regression)
mir_model.fit(train_x, train_y.ravel())
mir_filter = data.columns.values[np.argsort(mir_model.scores_)[:50]]

mir_model = SelectKBest(mutual_info_regression)
mir_model.fit(train_x, train_y_2.ravel())
mir_filter_2 = data.columns.values[np.argsort(mir_model.scores_)[:50]]
mir_filter = np.union1d(mir_filter, mir_filter_2)

train_y[train_y > 0] = 1
train_y[train_y <= 0] = 0
chi2_model = SelectKBest(chi2)
chi2_model.fit(train_x, train_y.ravel())
chi2_filter = data.columns.values[np.argsort(chi2_model.scores_)[:50]]

In [15]:
stage_1 = np.union1d(np.union1d(np.union1d(rr_filter, fr_filter), mir_filter), chi2_filter)

In [16]:
# Feature Selection 2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

train_x = df.values
train_y = df[label].values
train_y_2 = df["Adj_Close"].values

train_x = MinMaxScaler().fit_transform(train_x)
train_y = MinMaxScaler().fit_transform(train_y.reshape(-1, 1))
train_y_2 = MinMaxScaler().fit_transform(train_y_2.reshape(-1, 1))

linear = LinearRegression()  # less computational
rfe_linear = RFE(estimator=linear, n_features_to_select=1, step=1)
rfe_linear.fit(train_x, train_y.ravel())
rfe_linear_filter = data.columns.values[(rfe_linear.ranking_ - 1)[:50]]

linear = LinearRegression()  # less computational
rfe_linear = RFE(estimator=linear, n_features_to_select=1, step=1)
rfe_linear.fit(train_x, train_y_2.ravel())
rfe_linear_filter_2 = data.columns.values[(rfe_linear.ranking_ - 1)[:50]]

rfe_linear_filter = np.union1d(rfe_linear_filter, rfe_linear_filter_2)

In [17]:
# Feature Selection 3
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor

train_x = df.values
train_y = df[label].values
train_y_2 = df["Adj_Close"].values

train_x = MinMaxScaler().fit_transform(train_x)
train_y = MinMaxScaler().fit_transform(train_y.reshape(-1, 1))
train_y_2 = MinMaxScaler().fit_transform(train_y_2.reshape(-1, 1))

tree = RandomForestRegressor().fit(train_x, train_y.ravel())
model = SelectFromModel(tree, prefit=True)
sfm_filter = data.columns.values[np.where(model.get_support() == True)[0]]

tree = RandomForestRegressor().fit(train_x, train_y_2.ravel())
model = SelectFromModel(tree, prefit=True)
sfm_filter_2 = data.columns.values[np.where(model.get_support() == True)[0]]

sfm_filter = np.union1d(sfm_filter, sfm_filter_2)

In [18]:
final_selection = np.union1d(np.union1d(stage_1, rfe_linear_filter), sfm_filter)

In [19]:
np.setdiff1d(np.array(list(fields_dict.keys())), final_selection)

array(['Adjclose_EMA20', 'Adjclose_EMA3', 'Adjclose_EMA5', 'Adjclose_MA3',
       'Adjclose_MA5', 'Adjclose_Mad10', 'Adjclose_Slope20',
       'Adjclose_Slope3', 'Adjclose_WMA20', 'Adjclose_WMA3',
       'Close_EMA10', 'Close_MA5', 'Close_Mad10', 'Close_Slope20',
       'Close_Slope3', 'Factor', 'High_EMA3', 'High_EMA5', 'High_Mad3',
       'High_Slope10', 'High_Slope20', 'High_Slope3', 'High_Slope5',
       'High_WMA5', 'Low_EMA3', 'Low_Slope20', 'Low_Slope3', 'Open_EMA3',
       'Open_Mad10', 'Open_Mad20', 'Open_Slope10', 'Open_Slope20',
       'Open_Slope5', 'Open_WMA3', 'Open_WMA5'], dtype='<U18')

In [20]:
processed_df = df[final_selection]
processed_df.to_csv("data/feat_eng_filtered.csv")

In [21]:
plt.figure(figsize=(100, 100))
sns.heatmap(processed_df.corr(), cmap="crest")
plt.tight_layout()
plt.show()